<a href="https://colab.research.google.com/github/ThatGuyDV/CNNapproches/blob/master/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import getpass

In [3]:
!pip install git+https://github.com/Kaggle/kaggle-api.git --upgrade
import os
credentials = {"username":"dv1453","key":getpass.getpass()}
os.environ['KAGGLE_USERNAME']=credentials["username"]
os.environ['KAGGLE_KEY']=credentials["key"]

  Cloning https://github.com/Kaggle/kaggle-api.git to /tmp/pip-req-build-7k8f0_b_
  Running command git clone -q https://github.com/Kaggle/kaggle-api.git /tmp/pip-req-build-7k8f0_b_
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=78385 sha256=41ee4ccde2a4a96e3f61ef1d0ed1dc8cf0b0672184dda3eb595803903f4e297e
  Stored in directory: /tmp/pip-ephem-wheel-cache-op0zop_x/wheels/2f/2c/1d/18a38a6b617c2c338bdcbf409f3ccc2452799db7dca965d475
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6
··········


In [4]:
!kaggle datasets download -d alxmamaev/flowers-recognition --force

 96% 433M/450M [00:08<00:00, 44.9MB/s]
100% 450M/450M [00:08<00:00, 53.8MB/s]
